# Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile and train the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, 
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"\nTest accuracy: {test_acc:.4f}")

# Function to classify a single image
def classify_digit(model, image):
    prediction = model.predict(image.reshape(1, 28, 28, 1))
    return prediction.argmax()

# Example usage
import numpy as np
sample_image = test_images[0]
predicted_digit = classify_digit(model, sample_image)
print(f"The model predicts this image is a: {predicted_digit}")


# Result Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Visualize training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Visualize sample predictions
n_samples = 10
sample_indices = np.random.choice(len(test_images), n_samples, replace=False)

plt.figure(figsize=(20, 4))
for i, idx in enumerate(sample_indices):
    plt.subplot(2, n_samples, i + 1)
    plt.imshow(test_images[idx].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title(f"True: {test_labels[idx]}")
    
    plt.subplot(2, n_samples, i + 1 + n_samples)
    prediction = model.predict(test_images[idx].reshape(1, 28, 28, 1))
    predicted_label = np.argmax(prediction)
    plt.imshow(test_images[idx].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title(f"Pred: {predicted_label}")

plt.tight_layout()
plt.show()

# Create and visualize confusion matrix
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(test_labels, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Visualize model architecture
from tensorflow.keras.utils import plot_model

plt.figure(figsize=(12, 12))
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
plt.imshow(plt.imread('model_architecture.png'))
plt.axis('off')
plt.title('Model Architecture')
plt.show()

# Visualize feature maps
layer_outputs = [layer.output for layer in model.layers if 'conv2d' in layer.name]
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

sample_image = test_images[0]
activations = activation_model.predict(sample_image.reshape(1, 28, 28, 1))

for i, activation in enumerate(activations):
    plt.figure(figsize=(15, 8))
    for j in range(min(32, activation.shape[-1])):
        plt.subplot(4, 8, j+1)
        plt.imshow(activation[0, :, :, j], cmap='viridis')
        plt.axis('off')
    plt.suptitle(f'Feature Maps for Convolutional Layer {i+1}')
    plt.tight_layout()
    plt.show()